# Разбор простого примера с habr.ru

In [20]:
from yargy import rule, and_, or_, Parser
from yargy.predicates import gte, lte, caseless, normalized, dictionary
from yargy.interpretation import fact
from datetime import date

In [2]:
DAY = and_(
    gte(1),
    lte(31)
)
MONTH = and_(
    gte(1),
    lte(12)
)
YEAR = and_(
    gte(1),
    lte(2018)
)
DATE = rule(
    YEAR,
    '-',
    MONTH,
    '-',
    DAY
)

parser = Parser(DATE)

text = '''
    2018-02-23,
    2015-12-31;
    8 916 364-12-01
'''

for match in parser.findall(text):
    print(match.span, [x.value for x in match.tokens])

[5, 15) ['2018', '-', '02', '-', '23']
[21, 31) ['2015', '-', '12', '-', '31']
[43, 52) ['364', '-', '12', '-', '01']


In [5]:
MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}

MONTH_NAME = dictionary(MONTHS)

YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(normalized('год'))
)

DATE = or_(
    rule(
        YEAR,
        '-',
        MONTH,
        '-',
        DAY
    ),
    rule(
        DAY,
        MONTH_NAME,
        YEAR,
        YEAR_WORDS.optional()
    )
)

parser = Parser(DATE)

text = '''
    8 января 2014 года, 15 июня 2001 г.,
    31 февраля 2018
'''

for match in parser.findall(text):
    print(match.span, [x.value for x in match.tokens])

[5, 23) ['8', 'января', '2014', 'года']
[25, 40) ['15', 'июня', '2001', 'г', '.']
[46, 61) ['31', 'февраля', '2018']


In [10]:
match = parser.match('05 февраля 2011 года')
match.tree
# match

Tree(Node(BNFRule([Production([AndPredicate([gte(1), lte(2018)]),
                               eq('-'),
                               AndPredicate([gte(1), lte(12)]),
                               eq('-'),
                               AndPredicate([gte(1), lte(31)])],
                              0),
                   Production([AndPredicate([gte(1), lte(31)]),
                               DictionaryPredicate({'август',
                                                    'августа',
                                                    'апрель',
                                                    'декабрь',
                                                    'июль',
                                                    'июнь',
                                                    'май',
                                                    'март',
                                                    'марта',
                                                    'мая',
                  

In [19]:
Date = fact(
    'Date',
    ['year', 'month', 'day']
)


DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day
)
MONTH = and_(
    gte(1),
    lte(12)
).interpretation(
    Date.month
)
YEAR = and_(
    gte(1),
    lte(2018)
).interpretation(
    Date.year
)
MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month
)
DATE = or_(
    rule(YEAR, '-', MONTH, '-', DAY),
    rule(
        DAY,
        MONTH_NAME,
        YEAR,
        YEAR_WORDS.optional()
    )
).interpretation(Date)

# match = parser.match('05 февраля 2011 года')
# match

parser = Parser(DATE)

text = '''8 января 2014 года, 2018-12-01, 1 февраля 2014 года'''

for match in parser.findall(text):
    print(match.fact)

Date(year='2014', month='января', day='8')
Date(year='2018', month='12', day='01')
Date(year='2014', month='февраля', day='1')


In [21]:
Date = fact(
    'Date',
    ['year', 'month', 'day']
)

class Date(Date):
    @property
    def as_datetime(self):
        return date(self.year, self.month, self.day)


MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'мая': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}


DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day.custom(int)
)
MONTH = and_(
    gte(1),
    lte(12)
).interpretation(
    Date.month.custom(int)
)
YEAR = and_(
    gte(1),
    lte(2018)
).interpretation(
    Date.year.custom(int)
)
MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month.normalized().custom(MONTHS.__getitem__)
)
DATE = or_(
    rule(YEAR, '-', MONTH, '-', DAY),
    rule(
        DAY,
        MONTH_NAME,
        YEAR,
        YEAR_WORDS.optional()
    )
).interpretation(Date)

parser = Parser(DATE)
text = '''8 января 2014 года, 2018-12-01'''
for match in parser.findall(text):
    record = match.fact
    print(record, repr(record.as_datetime))

Date(year=2014, month=1, day=8) datetime.date(2014, 1, 8)
Date(year=2018, month=12, day=1) datetime.date(2018, 12, 1)
